# **Face Mask Classification** 😷

## **0.** Settings

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Unzip
!unzip '/content/drive/MyDrive/Face Mask/dataset.zip'

# Get the directory
cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'dataset')

In [21]:
# Libraries
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from IPython.display import Image, display
from tensorflow.keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import seaborn as sns
import numpy as np
import math
import cv2

# Warnings off
import warnings
warnings.filterwarnings("ignore")

# Split in train and validation
img_h = 256
img_w = 256
batch_size = 32

train_dataset = image_dataset_from_directory(
    dataset_dir,
    shuffle          = True,
    batch_size       = batch_size,
    image_size       = (img_h, img_w),
    validation_split = 0.3,
    subset           = 'training',
    seed             = 420)

validation_dataset = image_dataset_from_directory(
    dataset_dir,
    shuffle          = True,
    batch_size       = batch_size,
    image_size       = (img_h, img_w),
    validation_split = 0.3,
    subset           = 'validation',
    seed             = 420)

classes = train_dataset.class_names
num_classes = len(classes)

Found 5614 files belonging to 3 classes.
Using 3930 files for training.
Found 5614 files belonging to 3 classes.
Using 1684 files for validation.


## **1.** Model 1: **EfficientNetB3**

In [22]:
base_model = tf.keras.applications.EfficientNetB3(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Freeze the first 20 layers (out of 384)
for layer in base_model.layers[:20]:
    layer.trainable = False

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_01.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

43950080/43941136 [==============================] - 0s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.2838 - accuracy: 0.7308
Epoch 1: val_loss improved from inf to 5.12409, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_01.hdf5
123/123 [==============================] - 119s 735ms/step - loss: 1.2838 - accuracy: 0.7308 - val_loss: 5.1241 - val_accuracy: 0.6271 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.7623 - accuracy: 0.8776
Epoch 2: val_loss improved from 5.12409 to 1.29978, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_01.hdf5
123/123 [==============================] - 86s 697ms/step - loss: 0.7623 - accuracy: 0.8776 - val_loss: 1.2998 - val_accuracy: 0.8082 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.5123 - accuracy: 0.9214
Epoch 3: val_loss improved from 1.29978 to 0.65359, saving model to /content/drive/MyDrive/Face Mask/mode

## **2.** Model 2: **Xception**

In [23]:
base_model = tf.keras.applications.Xception(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Freeze the first 20 layers (out of 132)
for layer in base_model.layers[:20]:
    layer.trainable = False

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_02.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

83697664/83683744 [==============================] - 0s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.3308 - accuracy: 0.5036
Epoch 1: val_loss improved from inf to 203.54181, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_02.hdf5
123/123 [==============================] - 88s 657ms/step - loss: 1.3308 - accuracy: 0.5036 - val_loss: 203.5418 - val_accuracy: 0.3361 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.6521 - accuracy: 0.7201
Epoch 2: val_loss improved from 203.54181 to 0.98755, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_02.hdf5
123/123 [==============================] - 79s 635ms/step - loss: 0.6521 - accuracy: 0.7201 - val_loss: 0.9875 - val_accuracy: 0.6164 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.4934 - accuracy: 0.7908
Epoch 3: val_loss did not improve from 0.98755
123/123 [==============================] - 76s 611ms/

## **3.** Model 3: **ResNet50V2**

In [24]:
base_model = tf.keras.applications.ResNet50V2(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Freeze the first 20 layers (out of 190)
for layer in base_model.layers[:20]:
    layer.trainable = False

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_03.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

94683136/94668760 [==============================] - 0s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.4782 - accuracy: 0.5237
Epoch 1: val_loss improved from inf to 5.43553, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_03.hdf5
123/123 [==============================] - 63s 443ms/step - loss: 1.4782 - accuracy: 0.5237 - val_loss: 5.4355 - val_accuracy: 0.3818 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.8876 - accuracy: 0.7028
Epoch 2: val_loss improved from 5.43553 to 1.92032, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_03.hdf5
123/123 [==============================] - 52s 413ms/step - loss: 0.8876 - accuracy: 0.7028 - val_loss: 1.9203 - val_accuracy: 0.6419 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.6632 - accuracy: 0.7593
Epoch 3: val_loss did not improve from 1.92032
123/123 [==============================] - 49s 393ms/step -

## **4.** Model 4: **InceptionV3**

In [25]:
base_model = tf.keras.applications.InceptionV3(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_04.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

87924736/87910968 [==============================] - 1s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.3064 - accuracy: 0.6830
Epoch 1: val_loss improved from inf to 288.02151, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_04.hdf5
123/123 [==============================] - 65s 444ms/step - loss: 1.3064 - accuracy: 0.6830 - val_loss: 288.0215 - val_accuracy: 0.3379 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.7625 - accuracy: 0.8132
Epoch 2: val_loss improved from 288.02151 to 0.81493, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_04.hdf5
123/123 [==============================] - 52s 414ms/step - loss: 0.7625 - accuracy: 0.8132 - val_loss: 0.8149 - val_accuracy: 0.7494 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.5810 - accuracy: 0.8494
Epoch 3: val_loss did not improve from 0.81493
123/123 [==============================] - 48s 382ms/

## **5.** Model 5: **InceptionResNetV2**

In [26]:
base_model = tf.keras.applications.InceptionResNetV2(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_05.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

219070464/219055592 [==============================] - 1s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.1692 - accuracy: 0.7127
Epoch 1: val_loss improved from inf to 3.44472, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_05.hdf5
123/123 [==============================] - 138s 962ms/step - loss: 1.1692 - accuracy: 0.7127 - val_loss: 3.4447 - val_accuracy: 0.3670 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.6774 - accuracy: 0.8491
Epoch 2: val_loss did not improve from 3.44472
123/123 [==============================] - 108s 872ms/step - loss: 0.6774 - accuracy: 0.8491 - val_loss: 111.8034 - val_accuracy: 0.3385 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.5445 - accuracy: 0.8455
Epoch 3: val_loss improved from 3.44472 to 0.71814, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_05.hdf5
123/123 [==============================] - 117s 947ms

## **6.** Model 6: **DenseNet201**

In [27]:
base_model = tf.keras.applications.DenseNet201(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_06.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

74850304/74836368 [==============================] - 0s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.0441 - accuracy: 0.7008
Epoch 1: val_loss improved from inf to 2.76580, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_06.hdf5
123/123 [==============================] - 143s 953ms/step - loss: 1.0441 - accuracy: 0.7008 - val_loss: 2.7658 - val_accuracy: 0.4537 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.8020
Epoch 2: val_loss did not improve from 2.76580
123/123 [==============================] - 101s 817ms/step - loss: 0.6942 - accuracy: 0.8020 - val_loss: 4.9257 - val_accuracy: 0.3361 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.5461 - accuracy: 0.8425
Epoch 3: val_loss improved from 2.76580 to 1.68904, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_06.hdf5
123/123 [==============================] - 107s 863ms/ste

## **7.** Model 7: **EfficientNetV2S**

In [28]:
base_model = tf.keras.applications.EfficientNetV2S(
    include_top = False,
    weights     = "imagenet",
    input_shape = (img_h, img_w, 3))

for layer in base_model.layers:
    layer.trainable = True

# Build the model
x = tf.identity(base_model.output)
x = tf.keras.layers.AveragePooling2D(pool_size=(5,5))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(base_model.input, x)

# Compile the model
num_epochs    = 500
optimizer     = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
metrics       = ['accuracy']
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Save the model
filepath = '/content/drive/MyDrive/Face Mask/' + 'model_ensamble_07.hdf5'

# Callbacks
Early_Stopping = tf.keras.callbacks.EarlyStopping(
    monitor              = "val_loss", 
    restore_best_weights = False, 
    verbose              = 1, 
    patience             = 10)

Learning_Rate_Adapter = tf.keras.callbacks.ReduceLROnPlateau(
    monitor  = 'val_loss', 
    factor   = 0.2, 
    patience = 3, 
    verbose  = 1, 
    mode     = 'auto')

Best_model_save = tf.keras.callbacks.ModelCheckpoint(
    filepath       = filepath, 
    save_best_only = True, 
    monitor        = 'val_loss', 
    mode           = 'min', 
    verbose        = True)
 
callback = [Early_Stopping, Learning_Rate_Adapter, Best_model_save]

# Train the model
model.fit(train_dataset, epochs = num_epochs, validation_data = validation_dataset, callbacks = callback)

82436096/82420632 [==============================] - 0s 0us/step
Epoch 1/500
123/123 [==============================] - ETA: 0s - loss: 1.1797 - accuracy: 0.7491
Epoch 1: val_loss improved from inf to 0.97487, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_07.hdf5
123/123 [==============================] - 111s 751ms/step - loss: 1.1797 - accuracy: 0.7491 - val_loss: 0.9749 - val_accuracy: 0.7880 - lr: 0.0010
Epoch 2/500
123/123 [==============================] - ETA: 0s - loss: 0.6557 - accuracy: 0.8865
Epoch 2: val_loss improved from 0.97487 to 0.62326, saving model to /content/drive/MyDrive/Face Mask/model_ensamble_07.hdf5
123/123 [==============================] - 88s 714ms/step - loss: 0.6557 - accuracy: 0.8865 - val_loss: 0.6233 - val_accuracy: 0.8890 - lr: 0.0010
Epoch 3/500
123/123 [==============================] - ETA: 0s - loss: 0.4912 - accuracy: 0.8964
Epoch 3: val_loss improved from 0.62326 to 0.42456, saving model to /content/drive/MyDrive/Face Mask/mode